In [25]:
import pandas as pd
import os
from hcc_cal.commit import Mining
from environment import (
    PROJECTS
)

for project in PROJECTS:
    baseline_data = pd.read_csv(f"../data/dataset/baseline/{project}.csv", index_col=0)
    hcc_data = pd.read_csv(f"../data/dataset/hcc/{project}.csv", index_col=0)
    hcc_data.rename(columns={"ENT": "entropy"}, inplace=True)
    assert(set(baseline_data.index) == set(hcc_data.index))
    data = pd.concat([baseline_data, hcc_data.drop(labels=['buggy', 'repo', 'target', 'project', 'date', 'gap', 'Unnamed: 0'], axis=1)], axis=1)
    # print(data.columns)
    
    for commit_hash in data.index:
        commit = Mining.load("../data/cache", project, commit_hash)
        methods_len = len(commit.methods_after)
        asts = {}
        will_remove = set()
        for method in commit.methods_after:
            representation = ""
            for path, node in method.ast:
                representation += node.__repr__()
            asts[method.signature] = representation


        for method in commit.methods_before:
            representation = ""
            for path, node in method.ast:
                representation += node.__repr__()
            
            if asts[method.signature] == representation:
                # print(f"https://github.com/apache/{project}/commit/{commit_hash}")
                will_remove.add(method)

        for method in will_remove:
            commit.methods_after.remove(method)
            commit.methods_before.remove(method)

        if len(commit.methods_after) == 0:
            print(f"Empty commit {commit_hash}")
            # Remove commit from cache
            os.remove(f"../data/cache/{project}/{commit_hash}.pkl")
            # Remove commit from data
            data.drop(index=commit_hash, inplace=True)
        elif methods_len != len(commit.methods_after):
            # print(f"Removed {methods_len - len(commit.methods_after)} methods from {commit_hash}")
            Mining.save(commit, "../data/cache")

    data.to_csv(f"../data/dataset/filtered/{project}.csv")
    print(f"Filtered {project}")

            


Empty commit f051364d6b1e2c9c2b7aafc4066770a7ea6e2825
Empty commit 10abf6727ac026ac622355893e995646da678acb
Empty commit 4bff78d3b42de8002785652f8cea44d66b92306e
Empty commit 5477bc007fc85e2bbf37fd277058e7b8254792fc
Empty commit e11f786b9306642e67f6004e4f5fef2fabe7cc92
Empty commit 90beb3fb7dc2fd7aad71632a3102fe6f1dbdc5c6
Empty commit cf70ce97ee338135b5c5aaa8c7fa20eefa0d7222
Empty commit df2c7bdad42f6faa1ff89e4133f2fe7fc01150ec
Empty commit e35c39bbcf7b11755990502d8ac0167eb4e02cc2
Empty commit 94b9d551b7091155edd9fb28fba792ba4c58a826
Empty commit ff9b80040b187bc3baa0c8604d86f52808c48221
Empty commit a52c014648f47a7a9c2c78d990079757bbb35e76
Empty commit 90104943b28cac1a95b037fae8e338ffe04398fd
Empty commit 93f686c5cf13ee555ff76941f6825c19ffb0c3e3
Empty commit 960186af89d0fb6601ba147c5aa7216cc769aa7f
Empty commit 4b8cd46a005c5ff627dba04ad5205acd05354205
Empty commit 02c1e6d8f21d3bfc50154815ffb2f5caef05573c
Filtered activemq
Empty commit 3fdfe19f79229e5dc4aea2e4d110fbe399a21fc7
Empty comm

In [26]:
for project in PROJECTS:
    data = pd.read_csv(f"../data/dataset/filtered/{project}.csv", index_col=0)
    data["target"] = "not_yet"
    data.to_csv(f"../data/dataset/filtered/{project}.csv")